In [1]:
%%capture

!pip install "langchain==0.2.11"
!pip install "langchain-core==0.2.43"
!pip install langchain-google-genai

# Importing libraries

In [20]:
# Import Basic libraries
import os
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv

# Formatting
from IPython.display import Markdown, display

# Import Google Generative AI
from langchain_google_genai import ChatGoogleGenerativeAI 

# LangChain Components
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableSequence

# usually we start loading with environment variables
load_dotenv(override = True)

True

# API Initialization

In [4]:
google_api_key = os.getenv("GOOGLE_API_KEY")

In [15]:
def llm_model(prompt_txt, params=None):

    model_id = "gemini-2.0-flash"

    default_params = {
        "temperature" : 0,
        "max_tokens" : None, 
        "timeout" : None, 
        "max_retries" : 2
    }

    if params:
        default_params.update(params)

    # Create LLM directly
    llm = ChatGoogleGenerativeAI(
        model = model_id, 
        temperature = default_params["temperature"], 
        max_tokens = default_params["max_tokens"], 
        timeout = default_params["timeout"], 
        max_retries = default_params["max_retries"]
    )

    response = llm.invoke(prompt_txt)

    return response

In [14]:
ai_msg = llm_model(prompt_txt = [("system", "You are a helpful assistant that translates English to French. Translate the user sentense"), 
    ("human", "I love programming")])
print(ai_msg)    

Unexpected argument 'params' provided to ChatGoogleGenerativeAI.


content="J'adore la programmation." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--3d09e71f-8138-4d60-91b5-f8d68b74b030-0' usage_metadata={'input_tokens': 19, 'output_tokens': 7, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}}


# Prompt Engineering 

#### 1. Basic Prompt

A basic prompt is the simplest form of prompting, where you provide a short text or phrase to the model without any special formatting or instructions. The model generates a continuation based on patterns it has learned during training. Basic prompts are useful for exploring the model's capabilities and understanding how it naturally responds to minimal input

In [23]:
params = {
    "max_tokens" : 128, 
    "temperature" : 0.5, 
    "max_retries" : 2
}

prompt = "The wind is "

response = llm_model(prompt, params)
print(f"prompt : {prompt} \n")
print(f"response : {display(Markdown(response.content))} \n")

prompt : The wind is  



The wind is **blowing**.

response : None 



In [24]:
prompts = [
    "The future of artificial intellgence is",
    "Once upon a time in a distant galaxy",
    "The benefits of sustainable energy include"
]

for prompt in prompts:
    response = llm_model(prompt, params)
    print(f"prompt : {prompt} \n")
    print(f"response : {display(Markdown(response.content))} \n")

prompt : The future of artificial intellgence is 



The future of artificial intelligence is a topic of much discussion and speculation, with potential impacts across nearly every aspect of human life. Here's a breakdown of some key areas:

**Potential Positive Impacts:**

*   **Automation and Efficiency:** AI can automate repetitive tasks, freeing up humans for more creative and strategic work. This could lead to increased productivity and economic growth.
*   **Healthcare Advancements:** AI can assist in diagnosis, drug discovery, personalized medicine, and robotic surgery, potentially leading to better patient outcomes and more efficient healthcare systems.
*   **Improved Decision-Making:** AI algorithms can analyze vast amounts of data

response : None 

prompt : Once upon a time in a distant galaxy 



Once upon a time in a distant galaxy, far, far beyond the swirling nebulae of known space, existed a planet called Xylos. Xylos wasn't a typical planet of verdant forests or shimmering oceans. Instead, it was a world sculpted entirely from crystal. Towering spires of amethyst pierced the cerulean sky, while canyons of glittering quartz snaked across the landscape. The very air hummed with a low, resonant frequency, a song of the crystals themselves.

The inhabitants of Xylos were the Luminians, beings of pure light and energy, housed within crystalline shells. They were not born

response : None 

prompt : The benefits of sustainable energy include 



The benefits of sustainable energy are numerous and span environmental, economic, and social aspects. Here's a breakdown:

**Environmental Benefits:**

*   **Reduced Greenhouse Gas Emissions:** This is the most significant benefit. Sustainable energy sources like solar, wind, hydro, and geothermal produce little to no greenhouse gases during operation, mitigating climate change.
*   **Improved Air Quality:**  Unlike fossil fuels, sustainable energy sources don't release harmful air pollutants like sulfur dioxide, nitrogen oxides, and particulate matter, leading to cleaner air and reduced respiratory problems.
*   **Water Conservation:** Many sustainable energy technologies, particularly solar and wind

response : None 



#### 2. Zero-Shot Prompts

Zero-shot prompting is a technique where the model performs a task without any examples or prior specific training. This approach tests the model's ability to understand instructions and apply its knowledge to a new context without demonstration. Zero-shot prompts typically include clear instructions about what the model should do, allowing it to leverage its pre-trained knowledge effectively.